In [1]:
# === COMPREHENSIVE EMERGENCE MAPPING - CORRECTED VERSION ===
import math, random, statistics, itertools, os, sys, time
from fractions import Fraction
from decimal import Decimal, getcontext
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import warnings
warnings.filterwarnings('ignore')

# ============= CONFIGURATION =============
OUTDIR = "/content/rigorous_emergence_mapping"
os.makedirs(OUTDIR, exist_ok=True)

# Reduced for reliable testing first
N_DIGITS = 2000       # Smaller for speed
N_SEEDS = 8           # Fewer seeds for testing
N_BOOTSTRAP = 50      # Smaller bootstrap
K_MAX = 8             # Focused range
L_MAX = 12            # Reduced dyadic range
SEED_BASE = 123456
getcontext().prec = max(60, N_DIGITS + 80)

print(f"🔬 COMPREHENSIVE EMERGENCE MAPPING (CORRECTED)")
print(f"📊 Digits: {N_DIGITS}, Seeds: {N_SEEDS}, Bootstrap: {N_BOOTSTRAP}")
print(f"⚡ Corrected sequence generation")

# ============= COMPREHENSIVE CONSTANT LIBRARY =============
def build_comprehensive_constants():
    """Build exhaustive constant library with all variations."""
    base_constants = {
        'fine_structure': Decimal("0.0072973525693"),
        'fine_structure_codata': Decimal(1) / Decimal("137.035999084"),
        'phi': (Decimal(1) + Decimal(5).sqrt()) / 2,
        'pi': Decimal(str(math.pi)),
        'e': Decimal(str(math.e)),
        'sqrt2': Decimal(str(math.sqrt(2))),
        'sqrt3': Decimal(str(math.sqrt(3))),
        'feigenbaum_delta': Decimal("4.6692016091029906718532038204662"),
        'feigenbaum_alpha': Decimal("2.5029078750958928222839028732182"),
        'euler_gamma': Decimal("0.5772156649015328606065120900824"),
        'catalan': Decimal("0.9159655941772190150546035149324"),
        'inv137': Decimal(1) / Decimal(137),
        'inv9': Decimal(1) / Decimal(9),
    }

    constants = {}

    # For each base constant, generate ALL variations
    for name, value in base_constants.items():
        if value > 0:
            constants[f'{name}'] = value
            constants[f'{name}_inv'] = Decimal(1) / value
            constants[f'{name}_neg'] = -value
            constants[f'{name}_sqrt'] = value.sqrt()
            constants[f'{name}_half'] = value / 2
            constants[f'{name}_double'] = value * 2

            # Scale variations (your key insight)
            for scale in [10, 100, 1000, 10000, 100000]:
                constants[f'{name}_div{scale}'] = value / scale
                constants[f'{name}_x{scale}'] = value * scale

            # Root variations (your inverse suggestion)
            if value > 0:
                constants[f'{name}_cuberoot'] = value ** (Decimal(1)/3)
                constants[f'{name}_ninthroot'] = value ** (Decimal(1)/9)

    # Remove None values
    constants = {k: v for k, v in constants.items() if v is not None}

    print(f"📚 Built comprehensive library: {len(constants)} constant variations")
    return constants

# ============= CORRECTED SEQUENCE GENERATION =============
def generate_sequences_safely(n_digits, seed_idx=0):
    """Generate all sequences efficiently with NO redundant calls."""

    print(f"\n🧬 Generating sequences ({n_digits} bits, seed {seed_idx})...")

    sequences = {}

    try:
        # 1. Generate F sequence using golden ratio method
        print(f"    Generating F sequence...")
        phi = (1 + math.sqrt(5)) / 2
        sequences['F'] = []
        for i in range(n_digits):
            bit = int((i + 1) * phi) - int(i * phi)
            sequences['F'].append(1 - bit)  # Complement for standard Fibonacci word
            if (i + 1) % 1000 == 0:
                print(f"      F: {i+1}/{n_digits} bits...")
        print(f"  ✅ F complete: {len(sequences['F'])} bits")

        # 2. Generate R from F (complement - NO regeneration)
        print(f"    Generating R sequence (complement of F)...")
        sequences['R'] = [1 - b for b in sequences['F']]
        print(f"  ✅ R complete: {len(sequences['R'])} bits")

        # 3. Generate TM sequence
        print(f"    Generating TM sequence...")
        sequences['TM'] = []
        for i in range(n_digits):
            sequences['TM'].append(bin(i + seed_idx % 1000).count("1") & 1)
            if (i + 1) % 1000 == 0:
                print(f"      TM: {i+1}/{n_digits} bits...")
        print(f"  ✅ TM complete: {len(sequences['TM'])} bits")

        # 4. Generate FIBBI sequence
        print(f"    Generating FIBBI sequence...")
        phi_decimal = (Decimal(1) + Decimal(5).sqrt()) / 2
        x = phi_decimal - 1

        if seed_idx > 0:
            perturbation = Decimal(seed_idx) / Decimal(10**10)
            x += perturbation
            x = x % 1

        sequences['FIBBI'] = []
        for i in range(n_digits):
            x *= 2
            if x >= 1:
                sequences['FIBBI'].append(1)
                x -= 1
            else:
                sequences['FIBBI'].append(0)
            if (i + 1) % 1000 == 0:
                print(f"      FIBBI: {i+1}/{n_digits} bits...")
        print(f"  ✅ FIBBI complete: {len(sequences['FIBBI'])} bits")

        # Verification
        for name, seq in sequences.items():
            density = sum(seq) / len(seq)
            print(f"  ✓ {name}: {len(seq)} bits, density {density:.3f}")

        # Verify complement identity
        if len(sequences['F']) == len(sequences['R']):
            complement_check = all(f + r == 1 for f, r in zip(sequences['F'], sequences['R']))
            print(f"  ✓ Complement identity: {complement_check}")

        return sequences

    except Exception as e:
        print(f"❌ Generation failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return None

# ============= MATHEMATICAL OPERATIONS =============
def digits_to_real(bits, B=10):
    """Convert bits to real number in base B."""
    B = Decimal(B)
    w = Decimal(1) / B
    v = Decimal(0)
    for b in bits:
        if b:
            v += w
        w /= B
    return v

def find_closest_constants(value, constants_lib, max_matches=5):
    """Find closest constants with ultra-high sensitivity."""
    matches = []
    for name, const_val in constants_lib.items():
        try:
            if const_val is not None and const_val != 0:
                diff = abs(value - const_val)
                rel_diff = diff / abs(const_val) if const_val != 0 else float('inf')
                matches.append((name, const_val, float(diff), float(rel_diff)))
        except:
            continue

    matches.sort(key=lambda x: x[2])
    return matches[:max_matches]

# ============= TIER 1: TRIADIC OPERATIONS =============
def tier1_triadic_scan(bits, B, constants_lib, k_max=K_MAX):
    """Enhanced triadic scan."""
    D = digits_to_real(bits, B)
    results = []

    for k in range(k_max + 1):
        m = Decimal(3) ** k
        estimate = (m / Decimal(1000)) * D

        matches = find_closest_constants(estimate, constants_lib)

        result = {
            'base': B, 'k': k, 'estimate': estimate, 'D_value': D,
            'triadic_multiplier': m
        }

        if matches:
            result.update({
                'best_match_name': matches[0][0],
                'best_match_value': matches[0][1],
                'best_match_error': matches[0][2],
                'best_match_rel_error': matches[0][3]
            })

        results.append(result)

    return results

# ============= COMPREHENSIVE ANALYSIS =============
def run_corrected_emergence_analysis():
    """Corrected main analysis pipeline."""

    # Build constants library
    constants_lib = build_comprehensive_constants()

    # Results containers
    all_tier1_results = []

    print(f"\n🔬 Starting analysis with {N_SEEDS} seeds...")

    for seed_idx in range(N_SEEDS):
        print(f"\n--- SEED {seed_idx + 1}/{N_SEEDS} ---")

        current_seed = SEED_BASE + seed_idx * 1000
        random.seed(current_seed)
        np.random.seed(current_seed)

        # Generate sequences safely
        sequences = generate_sequences_safely(N_DIGITS, seed_idx)

        if sequences is None:
            print(f"❌ Skipping seed {seed_idx} due to generation failure")
            continue

        # Analysis - start with key bases only
        for base in [10, 2, 3]:  # Focus on key bases first
            for seq_name, bits in sequences.items():
                print(f"    Analyzing {seq_name} base-{base}...")

                # Tier 1 analysis
                tier1_results = tier1_triadic_scan(bits, base, constants_lib)

                for result in tier1_results:
                    result.update({
                        'sequence': seq_name,
                        'seed_idx': seed_idx,
                        'seed_value': current_seed
                    })
                    all_tier1_results.append(result)

                # Report best result
                if tier1_results:
                    best = min(tier1_results, key=lambda x: x.get('best_match_error', float('inf')))
                    if 'best_match_error' in best:
                        print(f"      Best: {best['best_match_name']} (error: {best['best_match_error']:.2e})")

        print(f"  ✅ Seed {seed_idx + 1} complete")

    # Save results
    print(f"\n💾 Saving results...")
    df_tier1 = pd.DataFrame(all_tier1_results)
    df_tier1.to_csv(f"{OUTDIR}/tier1_corrected.csv", index=False)

    if len(df_tier1) > 0 and 'best_match_error' in df_tier1.columns:
        best_overall = df_tier1.loc[df_tier1['best_match_error'].idxmin()]
        print(f"\n🎯 BEST OVERALL RESULT:")
        print(f"   {best_overall['sequence']} base-{best_overall['base']} k={best_overall['k']}")
        print(f"   {best_overall['best_match_name']}")
        print(f"   Error: {best_overall['best_match_error']:.2e}")

    return df_tier1

# ============= EXECUTE =============
if __name__ == "__main__":
    start_time = time.time()

    try:
        print("🚀 STARTING CORRECTED EMERGENCE MAPPING...")

        # Run the corrected analysis
        df_tier1 = run_corrected_emergence_analysis()

        runtime = time.time() - start_time
        print(f"\n⏱️  Runtime: {runtime:.1f} seconds ({runtime/60:.1f} minutes)")
        print(f"\n🎉 CORRECTED ANALYSIS COMPLETE!")
        print(f"📁 Results saved to: {OUTDIR}")

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

🔬 COMPREHENSIVE EMERGENCE MAPPING (CORRECTED)
📊 Digits: 2000, Seeds: 8, Bootstrap: 50
⚡ Corrected sequence generation
🚀 STARTING CORRECTED EMERGENCE MAPPING...
📚 Built comprehensive library: 234 constant variations

🔬 Starting analysis with 8 seeds...

--- SEED 1/8 ---

🧬 Generating sequences (2000 bits, seed 0)...
    Generating F sequence...
      F: 1000/2000 bits...
      F: 2000/2000 bits...
  ✅ F complete: 2000 bits
    Generating R sequence (complement of F)...
  ✅ R complete: 2000 bits
    Generating TM sequence...
      TM: 1000/2000 bits...
      TM: 2000/2000 bits...
  ✅ TM complete: 2000 bits
    Generating FIBBI sequence...
      FIBBI: 1000/2000 bits...
      FIBBI: 2000/2000 bits...
  ✅ FIBBI complete: 2000 bits
  ✓ F: 2000 bits, density -0.618
  ✓ R: 2000 bits, density 1.618
  ✓ TM: 2000 bits, density 0.500
  ✓ FIBBI: 2000 bits, density 0.497
  ✓ Complement identity: True
    Analyzing F base-10...
      Best: catalan_div10000 (error: 6.06e-07)
    Analyzing R base-10..